In [1]:
! pip install mlflow boto3 awscli optuna xgboost imbalanced-learn lightgbm

In [2]:
import mlflow
mlflow.set_tracking_uri("http://98.84.31.179:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

2024/12/15 19:44:01 INFO mlflow.tracking.fluent: Experiment with name 'ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://dsmp-mlflow-bucket/287048950003727427', creation_time=1734272062530, experiment_id='287048950003727427', last_update_time=1734272062530, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
import mlflow
import mlflow.sklearn
import optuna

C:\Users\kunkaush1\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [8]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: BOW vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment']).astype('float32')
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_BOW_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for LightGBM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LightGBM"
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for LightGBM
run_optuna_experiment()


[I 2024-12-15 19:54:57,276] A new study created in memory with name: no-name-d1efaecd-3641-4fcd-a70e-0fd835ed6fa1


[LightGBM] [Info] Number of positive: 12616, number of negative: 12616
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034692 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6172
[LightGBM] [Info] Number of data points in the train set: 25232, number of used features: 967
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2024-12-15 19:54:57,627] Trial 0 finished with value: 0.7468294229549778 and parameters: {'n_estimators': 66, 'learning_rate': 0.0011196451134281725, 'max_depth': 10}. Best is trial 0 with value: 0.7468294229549778.


[LightGBM] [Info] Number of positive: 12616, number of negative: 12616
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036830 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6172
[LightGBM] [Info] Number of data points in the train set: 25232, number of used features: 967
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2024-12-15 19:54:58,152] Trial 1 finished with value: 0.7468294229549778 and parameters: {'n_estimators': 131, 'learning_rate': 0.00017442747394300466, 'max_depth': 10}. Best is trial 0 with value: 0.7468294229549778.
[I 2024-12-15 19:54:58,391] Trial 2 finished with value: 0.7232086239695624 and parameters: {'n_estimators': 61, 'learning_rate': 0.021880775655466634, 'max_depth': 3}. Best is trial 0 with value: 0.7468294229549778.


...

[LightGBM] [Info] Number of positive: 12616, number of negative: 12616
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041908 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6172
[LightGBM] [Info] Number of data points in the train set: 25232, number of used features: 967
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

[I 2024-12-15 19:55:15,855] Trial 29 finished with value: 0.8909321496512366 and parameters: {'n_estimators': 278, 'learning_rate': 0.031217256164490392, 'max_depth': 10}. Best is trial 13 with value: 0.9208941027266963.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 12616, number of negative: 12616
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031457 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6172
[LightGBM] [Info] Number of data points in the train set: 25232, number of used features: 967
[LightGBM] [Info] [binary:BoostFromScor

2024/12/15 19:55:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_SMOTE_BOW_Trigrams at: http://98.84.31.179:5000/#/experiments/287048950003727427/runs/07020be15996410abdb1ab2ff4e62c12
🧪 View experiment at: http://98.84.31.179:5000/#/experiments/287048950003727427
